# Multiagentes de IA ◦ Criador de Post para Instagram com Gemini

## Instalar e Importar Pacotes

In [1]:
%pip -q install google-genai
!pip install -q google-adk #framework de agentes do google

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [21]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

## Configurar a API Key do Google Gemini / Acesso ao Google Secret

In [3]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

## Configurar o cliente da SDK do Gemini

In [23]:
from google import genai

client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [27]:
# Lista os modelos disponíveis
for model in client.models.list():
  print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-pr

## Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

In [5]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

##  Função para exibir texto formatado em Markdown no Colab

In [6]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Agente 1: Pesquisador de Notícias

In [33]:
##########################################
# --- Agente 1: Pesquisador de Notícias --- #
##########################################
def agente_pesquisador(topico, data_de_hoje):

    pesquisador = Agent(
        name="agente_pesquisador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_pesquisador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(pesquisador, entrada_do_agente_pesquisador)
    return lancamentos

## Agente 2: Planejador de posts

In [34]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes pesquisador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

## Agente 3: Redator do Post

In [35]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

## Agente 4: Revisor de Qualidade

In [36]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Após revisar, sempre forneça um resumo da sua avaliação. Se o rascunho estiver bom, diga 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

## Executando as Tarefas

In [37]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes...\n")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_pesquisador(topico, data_de_hoje)
    print("\n--- 🔎 Resultado do Agente 1 (Pesquisador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📋 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- ✔️ Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes...

❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: artistas que se destacam na cena de jazz contemporâneo
Maravilha! Vamos então criar o post sobre novidades em artistas que se destacam na cena de jazz contemporâneo

--- 🔎 Resultado do Agente 1 (Pesquisador) ---



> Para identificar artistas de destaque no jazz contemporâneo com lançamentos recentes, vou realizar algumas buscas no Google. O objetivo é encontrar notícias e artigos sobre os lançamentos de álbuns ou singles mais comentados e aclamados do último mês.
> 
> 
> Com base nas buscas realizadas, destaco alguns artistas e lançamentos que parecem promissores na cena do jazz contemporâneo em maio de 2025:
> 
> 1.  **Ambrose Akinmusire**: O álbum "Honey From a Winter Stone" (Nonesuch Records) tem sido apontado como um dos melhores álbuns de jazz de 2025 até o momento (fonte: [2], [15]).
> 
> 2.  **Gerald Clayton**: Seu terceiro álbum para a Blue Note, "Ones & Twos," explora territórios sonoros interessantes com a produção de Kassa Overall e influências de turntablism (fonte: [13]).
> 
> 3.  **Brian Charette**: "Borderless" (SteepleChase), o terceiro álbum solo de órgão Hammond B-3 de Charette, está sendo elogiado por suas linhas angulares e propulsão rítmica (fonte: [13]).
> 
> 4.  **Chief Adjuah (anteriormente Christian Scott)**: Reconhecido como um dos músicos mais inovadores do jazz contemporâneo, criador da "Stretch Music" (fonte: [23]).
> 
> 5.  **Samara Joy**: Vencedora do Grammy 2025 na categoria de jazz, tem se destacado com o álbum "Portrait" (2024) e apresentações ao vivo (fonte: [21]).
> 
> É importante notar que muitos festivais de jazz estão programados para os próximos meses, apresentando uma variedade de artistas consagrados e emergentes (fontes: [1], [4], [5], [7], [9], [11], [12], [17], [18], [22]). Além disso, publicações especializadas como a JazzTimes continuam a destacar lançamentos importantes ao longo do ano (fonte: [13]).
> 


--------------------------------------------------------------

--- 📋 Resultado do Agente 2 (Planejador) ---



> Com base nas informações fornecidas e em minhas habilidades como planejador de conteúdo especializado em redes sociais, elaborei um plano para cada um dos artistas mencionados, destacando os pontos mais relevantes para um possível post e usando a ferramenta de busca para aprofundar o conhecimento sobre cada um.
> 
> **1. Ambrose Akinmusire:**
> 
> *   **Pontos Relevantes:**
>     *   Álbum "Honey From a Winter Stone" (Nonesuch Records) amplamente considerado um dos melhores de 2025.
>     *   Reconhecimento da crítica especializada.
>     *   Estilo inovador e abordagem única ao jazz contemporâneo.
> *   **Plano de Conteúdo:**
>     *   Introdução: Apresentar Ambrose Akinmusire como um dos principais nomes do jazz atual.
>     *   Destaque do álbum: Focar em "Honey From a Winter Stone", mencionando a gravadora e o impacto do álbum.
>     *   Repercussão: Citar críticas e prêmios recebidos (se houver).
>     *   Estilo musical: Descrever sua sonoridade, enfatizando a inovação.
>     *   Chamada para ação: Convidar os seguidores a ouvir o álbum e compartilhar suas impressões.
>     *   `google_search.search(queries=["Honey From a Winter Stone review", "Ambrose Akinmusire estilo musical"])`
> 
> **2. Gerald Clayton:**
> 
> *   **Pontos Relevantes:**
>     *   Álbum "Ones & Twos" (Blue Note) explora novas sonoridades com produção de Kassa Overall.
>     *   Influências de turntablism no jazz.
>     *   Evolução na carreira do artista.
> *   **Plano de Conteúdo:**
>     *   Introdução: Apresentar Gerald Clayton como um artista em constante evolução.
>     *   Destaque do álbum: "Ones & Twos" e a colaboração com Kassa Overall.
>     *   Inovação: Explorar a influência do turntablism e como isso se manifesta no jazz.
>     *   Blue Note Records: Mencionar a importância da gravadora na cena do jazz.
>     *   Chamada para ação: Incentivar os seguidores a descobrir o álbum e a comentar sobre a fusão de estilos.
>     *   `google_search.search(queries=["Gerald Clayton Ones & Twos review", "Gerald Clayton turntablism jazz"])`
> 
> **3. Brian Charette:**
> 
> *   **Pontos Relevantes:**
>     *   Álbum "Borderless" (SteepleChase) focado no órgão Hammond B-3.
>     *   Reconhecimento pelas linhas angulares e ritmo pulsante.
>     *   Contribuição para a música de órgão no jazz.
> *   **Plano de Conteúdo:**
>     *   Introdução: Apresentar Brian Charette como um mestre do órgão Hammond B-3.
>     *   Destaque do álbum: "Borderless" e a sonoridade única do órgão.
>     *   Estilo musical: Descrever as linhas angulares e o ritmo que o caracterizam.
>     *   SteepleChase Records: Mencionar a relevância da gravadora no jazz.
>     *   Chamada para ação: Perguntar aos seguidores se eles apreciam o som do órgão no jazz e convidá-los a ouvir "Borderless".
>     *   `google_search.search(queries=["Brian Charette Borderless review", "Brian Charette órgão Hammond B-3 jazz"])`
> 
> **4. Chief Adjuah (anteriormente Christian Scott):**
> 
> *   **Pontos Relevantes:**
>     *   Pioneiro da "Stretch Music".
>     *   Um dos músicos mais inovadores do jazz contemporâneo.
>     *   Contribuições para a evolução do gênero.
> *   **Plano de Conteúdo:**
>     *   Introdução: Apresentar Chief Adjuah como um visionário do jazz.
>     *   Stretch Music: Explicar o conceito e a importância dessa inovação.
>     *   Legado: Destacar suas contribuições para a música.
>     *   Estilo musical: Descrever a sonoridade única e a visão vanguardista.
>     *   Chamada para ação: Convidar os seguidores a explorar a "Stretch Music" e a discutir o futuro do jazz.
>     *   `google_search.search(queries=["Chief Adjuah Stretch Music", "Chief Adjuah inovação jazz"])`
> 
> **5. Samara Joy:**
> 
> *   **Pontos Relevantes:**
>     *   Vencedora do Grammy 2025 na categoria de jazz.
>     *   Álbum "Portrait" (2024) aclamado.
>     *   Talento vocal e performances ao vivo marcantes.
> *   **Plano de Conteúdo:**
>     *   Introdução: Apresentar Samara Joy como uma das vozes mais promissoras do jazz.
>     *   Prêmio Grammy: Destacar a conquista e o reconhecimento da indústria.
>     *   Álbum "Portrait": Mencionar o sucesso do álbum e suas qualidades.
>     *   Talento vocal: Elogiar a técnica e a expressividade da cantora.
>     *   Chamada para ação: Convidar os seguidores a ouvir "Portrait" e a acompanhar a carreira de Samara Joy.
>     *   `google_search.search(queries=["Samara Joy Grammy 2025", "Samara Joy Portrait album review"])`
> 
> **Escolha do Tema Mais Relevante e Plano Detalhado:**
> 
> Considerando o impacto recente, o reconhecimento da crítica e o frescor da proposta, o tema mais relevante para um post seria **Samara Joy**.
> 
> **Plano Detalhado para Post sobre Samara Joy:**
> 
> 1.  **Título:** "Samara Joy: A Voz que Conquistou o Jazz e o Grammy"
> 
> 2.  **Introdução (2-3 frases):**
>     *   Apresentar Samara Joy como a mais recente sensação do jazz vocal.
>     *   Mencionar sua vitória no Grammy 2025 como um marco em sua carreira.
>     *   Destacar o álbum "Portrait" como um trabalho aclamado pela crítica e pelo público.
> 
> 3.  **Corpo do Post (4-6 parágrafos):**
>     *   **Grammy 2025:** Detalhar a importância do prêmio e como ele impulsionou a carreira de Samara Joy. Incluir citações da cantora sobre a premiação, se disponíveis.
>     *   **Álbum "Portrait":**
>         *   Analisar o álbum, mencionando as faixas de destaque e o estilo musical.
>         *   Citar trechos de resenhas que elogiem a voz e a interpretação de Samara Joy.
>     *   **Talento Vocal:**
>         *   Descrever as qualidades vocais de Samara Joy, como timbre, técnica e expressividade.
>         *   Compará-la com outras grandes cantoras de jazz, mostrando sua originalidade.
>     *   **Performances ao Vivo:**
>         *   Mencionar apresentações marcantes da cantora, destacando sua presença de palco e interação com o público.
>         *   Incluir vídeos ou trechos de shows, se possível.
> 
> 4.  **Chamada para Ação (1-2 frases):**
>     *   Convidar os seguidores a ouvir o álbum "Portrait" nas plataformas digitais.
>     *   Incentivar o público a acompanhar a carreira de Samara Joy e a compartilhar suas impressões sobre a cantora.
>     *   Perguntar qual a música preferida do álbum.
> 
> 5.  **Recursos Visuais:**
>     *   Foto de alta qualidade de Samara Joy recebendo o Grammy ou em uma performance.
>     *   Capa do álbum "Portrait".
>     *   Vídeos curtos de apresentações ao vivo.
> 
> 6.  **Hashtags:**
>     *   \#SamaraJoy \#JazzVocal \#Grammy2025 \#PortraitAlbum \#JazzContemporâneo \#NovaMúsica \#MúsicaDeQualidade
> 
> 7. **Google Search Keywords**
> * `google_search.search(queries=["Samara Joy entrevista Grammy 2025", "Samara Joy melhores músicas Portrait album", "Samara Joy próximos shows"])`
> 
> Este plano detalhado fornecerá um post completo e informativo sobre Samara Joy, destacando seus principais feitos e incentivando o público a conhecer sua música.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> Beleza! Com base no seu plano detalhado, aqui está um rascunho de post para o Instagram sobre Samara Joy:
> 
> ✨ Samara Joy: A Voz que Conquistou o Jazz e o Grammy! ✨
> 
> Preparem os ouvidos porque essa voz vai te encantar! 🎶 Samara Joy chegou para renovar o jazz vocal e já fez história ao levar o Grammy 2025 pra casa! 🏆
> 
> Seu álbum "Portrait" é uma obra-prima que conquistou crítica e público. Cada faixa é um deleite, mostrando o talento e a paixão dessa artista incrível. ✨
> 
> A vitória no Grammy não foi por acaso! Samara Joy tem um timbre único, técnica impecável e uma expressividade que arrepia. 🎤 Ela canta com a alma e encanta a todos com sua presença de palco.
> 
> E as performances ao vivo? De tirar o fôlego! 🤩 Samara Joy domina o palco e interage com o público de um jeito que te faz sentir parte da música.
> 
> Corre para ouvir "Portrait" nas plataformas digitais e prepare-se para se apaixonar! Qual sua música preferida do álbum? Conta pra gente nos comentários! 👇
> 
> Não perca de vista essa estrela! ✨ Samara Joy é o futuro do jazz e promete emocionar o mundo com sua voz.
> 
> \#SamaraJoy \#JazzVocal \#Grammy2025 \#PortraitAlbum \#JazzContemporâneo
> 


--------------------------------------------------------------

--- ✔️ Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------
